<a href="https://colab.research.google.com/github/joabsantanna/business_solution-automacao-email/blob/main/business_solution_automa%C3%A7%C3%A3o_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Problema de negócio**
___

Como parte da equipe de Analytics de uma grande marca de vestuário com mais de 25 lojas espalhadas em Shoppings de todo o Brasil, toda semana é necessário enviar para a diretoria um ranking atualizado com as 25 lojas contendo 3 informações:

- Faturamento de cada Loja
- Quantidade de Produtos Vendidos de cada Loja
- Ticket Médio dos Produto de cada Loja

Além disso, cada loja tem 1 gerente que precisa receber o resumo das informações da loja dele. Por isso, cada gerente deve receber no e-mail:

- Faturamento da sua loja
- Quantidade de Produtos Vendidos da sua loja
- Ticket Médio dos Produto da sua Loja

Esse relatório é sempre enviado como um resumo de todos os dados disponíveis no ano.

## **Proposta de resolução do problema**
___

Sequência de resolução em etapas:

- Etapa 1 - Importar e visualizar a Base de Dados para ver se precisamos fazer algum tratamentoa base de Dados

- Etapa 2 - Calcular os indicadores de todas as lojas:
  - Faturamento por Loja
  - Quantidade de Produtos Vendidos por Loja
  - Ticket Médio dos Produto por Loja

- Etapa 3 - Enviar e-mail para a diretoria

- Etapa 4 - Enviar e-mail para cada loja



### **Etapa 1 - Importar e visualizar a Base de Dados** 
_________

In [ ]:
# Importa bilbiotecas 
import pandas as pd

In [ ]:
# Leitura do dataset
tabelas_vendas = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Vendas.xlsx")
display(tabelas_vendas)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600
4,2,2019-01-02,Norte Shopping,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,Center Shopping Uberlândia,Short Listrado,2,102,204
100995,69996,2019-12-26,Center Shopping Uberlândia,Mochila,4,270,1080
100996,69996,2019-12-26,Center Shopping Uberlândia,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,Ribeirão Shopping,Camisa Listrado,1,108,108


### **Etapa 2 - Calculo dos indicadores de todas as lojas**
___





#### **Faturamento**

In [ ]:
tabela_faturamento = tabelas_vendas[["ID Loja", "Valor Final"]].groupby("ID Loja").sum()
tabela_faturamento = tabela_faturamento.sort_values(by="Valor Final", ascending=False)
display(tabela_faturamento)

,Valor Final
ID Loja,
Iguatemi Campinas,1762419
Shopping Vila Velha,1731167
Bourbon Shopping SP,1726110
Rio Mar Recife,1722766
Shopping SP Market,1721763
Palladium Shopping Curitiba,1721120
Norte Shopping,1711968
Ribeirão Shopping,1707122
Iguatemi Esplanada,1699681


#### **Quantidades de vendas**

In [ ]:
tabela_quantidade = tabelas_vendas[["ID Loja", "Quantidade"]].groupby("ID Loja").sum()
display(tabela_quantidade)

,Quantidade
ID Loja,
Bourbon Shopping SP,8863
Center Shopping Uberlândia,8627
Iguatemi Campinas,8935
Iguatemi Esplanada,8580
Norte Shopping,9014
Novo Shopping Ribeirão Preto,8751
Palladium Shopping Curitiba,9091
Parque Dom Pedro Shopping,8388
Passei das Águas Shopping,8618


#### **Ticket Médio dos produtos**

In [ ]:
ticket_medio = (tabela_faturamento["Valor Final"] / tabela_quantidade["Quantidade"]).to_frame()
ticket_medio = ticket_medio.rename(columns={0: "Ticket Medio"})
display(ticket_medio)

,Ticket Medio
ID Loja,
Bourbon Shopping SP,194.754598
Center Shopping Uberlândia,193.453228
Iguatemi Campinas,197.248909
Iguatemi Esplanada,198.098019
Norte Shopping,189.923231
Novo Shopping Ribeirão Preto,191.775226
Palladium Shopping Curitiba,189.321307
Parque Dom Pedro Shopping,194.519552
Passei das Águas Shopping,191.345324


### **Função enviar E-mail**


In [ ]:
import smtplib
import email.message

def enviar_email(nome_da_loja, tabela):

  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = f'''
  <p>Prezados,</p>
  <p>Segue relatório de vendas</p>
  {tabela.to_html()}
  <p>Qualquer dúvida estou à disposição</p>
  ''' 
  msg = email.message.Message()

  msg['Subject'] = f"Relatório de Vendas - {nome_da_loja}" 
  msg['From'] = 'email_remetente@gmail.com' 
  msg['To'] = 'email_destinatario@gmail.com' 
  password = "senha" 
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(corpo_email )
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')


### **Etapa 3 - Enviar e-mail para a diretoria**
____

In [ ]:
tabela_completa = tabela_faturamento.join(tabela_quantidade).join(ticket_medio)
#enviar_email("Diretoria", tabela_completa)

display(tabela_completa)

,Valor Final,Quantidade,Ticket Medio
ID Loja,,,
Iguatemi Campinas,1762419,8935,197.248909
Shopping Vila Velha,1731167,9224,187.680724
Bourbon Shopping SP,1726110,8863,194.754598
Rio Mar Recife,1722766,8863,194.377299
Shopping SP Market,1721763,8927,192.871401
Palladium Shopping Curitiba,1721120,9091,189.321307
Norte Shopping,1711968,9014,189.923231
Ribeirão Shopping,1707122,8825,193.441586
Iguatemi Esplanada,1699681,8580,198.098019


### **Etapa 4 - Enviar e-mail para cada loja**
___

In [ ]:
# Email eviado para uma única loja

loja = "Shopping Barra"

tabela_loja = tabelas_vendas.loc[tabelas_vendas["ID Loja"] == loja, ["ID Loja", "Quantidade", "Valor Final"]]
tabela_loja = tabela_loja.groupby("ID Loja").sum()
tabela_loja["Ticket Medio"] = tabela_loja["Valor Final"] / tabela_loja["Quantidade"]
enviar_email(loja, tabela_loja)

display(tabela_loja)

,Quantidade,Valor Final,Ticket Medio
ID Loja,,,
Shopping Barra,8638,1653103,191.375666


In [ ]:
# Email eviado para todas as lojas

lista_lojas = tabelas_vendas["ID Loja"].unique()

for loja in lista_lojas:

  tabela_loja = tabelas_vendas.loc[tabelas_vendas["ID Loja"] == loja, ["ID Loja", "Quantidade", "Valor Final"]]
  tabela_loja = tabela_loja.groupby("ID Loja").sum()
  tabela_loja["Ticket Medio"] = tabela_loja["Valor Final"] / tabela_loja["Quantidade"]
  enviar_email(loja, tabela_loja)

display(tabela_loja)

,Quantidade,Valor Final,Ticket Medio
ID Loja,,,
Salvador Shopping,8698,1646739,189.323868


In [ ]:
# Lista de lojas que receberão os e-mails

lista_lojas = tabelas_vendas["ID Loja"].unique()
print(lista_lojas)

['Iguatemi Esplanada' 'Norte Shopping' 'Rio Mar Shopping Fortaleza'
 'Shopping Barra' 'Shopping Ibirapuera' 'Iguatemi Campinas'
 'Shopping Center Leste Aricanduva' 'Passei das Águas Shopping'
 'Shopping Recife' 'Shopping Midway Mall' 'Bourbon Shopping SP'
 'Shopping Center Interlagos' 'Parque Dom Pedro Shopping'
 'Center Shopping Uberlândia' 'Shopping União de Osasco'
 'Shopping Eldorado' 'Shopping Vila Velha' 'Novo Shopping Ribeirão Preto'
 'Rio Mar Recife' 'Palladium Shopping Curitiba' 'Shopping SP Market'
 'Ribeirão Shopping' 'Shopping Iguatemi Fortaleza' 'Shopping Morumbi'
 'Salvador Shopping']
